In [1]:
pip install sastrawi

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd 
import string 
import nltk 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from snowballstemmer import stemmer

In [3]:
dataset_sg = pd.read_csv('data_fix.csv')

In [4]:
#Case Folding
data_baru = dataset_sg['Ulasan'].str.lower()
dataset_sg['Ulasan'] = data_baru

In [5]:
#Menghapus tag <br>
dataset_sg['Ulasan'] = dataset_sg['Ulasan'].replace("<br>", " ")

In [6]:
#Hapus angka dan simbol
def hapus_angka_simbol(data):
    data_bersih = re.sub('[^a-zA-Z]', ' ', data)
    return data_bersih

dataset_sg['Ulasan'] = dataset_sg['Ulasan'].apply(hapus_angka_simbol)

In [7]:
#baca slangword
slang_dict = pd.read_excel('sw.xlsm')

# Buat dictionary untuk mapping slangword dan standar
slang_map = dict(zip(slang_dict['slang'], slang_dict['formal']))

In [8]:
# Membuat fungsi untuk mencari dan mengganti slangword
def hapus_slangword(text):
    for key in slang_map:
        text = re.sub(rf"\b{key}\b", slang_map[key], text)
    return text

# Mengganti slangword di kolom tertentu
dataset_sg['Ulasan'] = dataset_sg['Ulasan'].apply(hapus_slangword)

In [9]:
#Tokenizing with unigram
def token_unigram(data):
    datas = word_tokenize(data)
    return [token for token in datas if len(token) > 1]

dataset_sg['Ulasan'] = dataset_sg['Ulasan'].apply(token_unigram)

In [10]:
#Remove Indonesian Stopword
nltk.download('stopwords')
indonesia_stop = set(stopwords.words('indonesian'))
indonesia_stop.discard('tidak')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rivaldo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
def hilang_stopword(data):
    delete_stop = [token for token in data if token not in indonesia_stop]
    return_data = ' '.join(delete_stop)
    return return_data
dataset_sg['Ulasan'] = dataset_sg['Ulasan'].apply(hilang_stopword)

In [12]:
def stemming(text):
    indo_stemmer = stemmer('indonesian')
    stemmed_text = ' '.join(indo_stemmer.stemWords(text.split()))
    return stemmed_text

dataset_sg['Ulasan'] = dataset_sg['Ulasan'].apply(stemming)

In [13]:
dataset_sg['Ulasan']

0                              sayang jalan masuk sempit
1             wisata bagus buang sampah sembarang sayang
2                    loket parkir biar tidak macet cepat
3      orang kota sibuk sawah pandang arik duduk sant...
4      wow alam pandang asri dingin kayak pandanga ca...
                             ...                        
669                               jalan utama baik mobil
670    panas banget baik pandang nya bagus air segar ...
671                               tutup pagi paka banget
672    bagus alam sayang nya batu alas kolam renang t...
673            bagus pandang sayang makan tidak variatif
Name: Ulasan, Length: 674, dtype: object

In [15]:
dataset_sg.to_csv('d_ns_t.csv', index=False)